In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
from dateutil.relativedelta import relativedelta

# data load
data1 = pd.read_excel("/Users/yj.noh/Desktop/train_data_2022.xlsx")
data2 = pd.read_excel("/Users/yj.noh/Desktop/train_data_2023.xlsx")

data = pd.concat([data1, data2], ignore_index=True)
data = data.rename(columns={"라이더수": "rider_cnt", "주문수": "order_cnt"})

# seoul
data = data[data["pick_rgn1_nm"] == "서울특별시"]
max_reg_date = data["reg_date"].max()
data = data[data["reg_date"] < max_reg_date]

data = data[data["hour_reg"].isin([9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23])]

# NA 채우기 - time table
data["reg_date"] = pd.to_datetime(data["reg_date"])
data["datetime"] = data["reg_date"] + pd.to_timedelta(data["hour_reg"], unit="h")

# weather
weather1 = pd.read_csv("/Users/yj.noh/Desktop/weather_2022.csv", encoding="cp949")
weather2 = pd.read_csv("/Users/yj.noh/Desktop/weather_2023.csv", encoding="cp949")
weather = pd.concat([weather1, weather2], ignore_index=True)
weather = weather.rename(columns={"기온..C.": "temp_c", "강수량.mm.": "rain_c", "적설.cm.": "snow_c", "일시": "date"})
weather["date"] = pd.to_datetime(weather["date"])
weather["hour"] = weather["date"].dt.hour
weather = weather[weather["hour"].isin([9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23])]

combined_data = pd.merge(data, weather[["date", "hour", "temp_c", "rain_c", "snow_c"]], left_on=["reg_date", "hour_reg"], right_on=["date", "hour"], how="left")
combined_data = combined_data.drop(columns=["date", "hour"])
combined_data["rain_c"] = combined_data["rain_c"].fillna(0)
combined_data["snow_c"] = combined_data["snow_c"].fillna(0)

# 변수 생성
combined_data["is_rain"] = np.where((combined_data["rain_c"] > 0) | (combined_data["snow_c"] > 0), 1, 0)

combined_data["month"] = combined_data["reg_date"].dt.month
combined_data["week"] = np.ceil(combined_data["reg_date"].dt.day / 7)


